In [1]:
import os
import torch
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from PIL import Image, UnidentifiedImageError
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
data_dir = '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages'

In [3]:
IGNORE_PATHS = [
    '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/11702.jpg',
    '/kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/666.jpg'
]

In [4]:
# Filter valid paths excluding those in IGNORE_PATHS
def get_filtered_image_paths(data_dir, ignore_paths):
    data_paths = []
    for folder in ['Cat', 'Dog']:
        folder_path = os.path.join(data_dir, folder)
        for filename in os.listdir(folder_path):
            filepath = os.path.join(folder_path, filename)
            if filepath not in ignore_paths:  # Exclude ignored paths
                try:
                    img = Image.open(filepath)
                    img.verify()  # Verify image integrity
                    data_paths.append(filepath)
                except (IOError, SyntaxError, Image.UnidentifiedImageError):
                    print(f"Skipping corrupt image: {filepath}")
    return data_paths

In [5]:
# Get filtered list of image paths
data_paths = get_filtered_image_paths(data_dir, IGNORE_PATHS)
print(f"Total valid images after filtering: {len(data_paths)}")

Skipping corrupt image: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Cat/Thumbs.db


/opt/conda/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Skipping corrupt image: /kaggle/input/microsoft-catsvsdogs-dataset/PetImages/Dog/Thumbs.db
Total valid images after filtering: 24998


In [6]:
# Define the custom dataset
class CustomDataset(Dataset):
    def __init__(self, data_path, transform=None):
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.data_path)

    def __getitem__(self, idx):
        img_path = self.data_path[idx]
        img = Image.open(img_path).convert("RGB")  # Ensure 3 channels
        if self.transform:
            img = self.transform(img)
        label = 0 if "Dog" in img_path else 1
        return img, label

In [7]:
# Define transformations
transformation_steps = transforms.Compose([
    transforms.Resize((370, 370)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor()
])

In [8]:
# Instantiate the dataset with filtered paths
dataset = CustomDataset(data_path=data_paths, transform=transformation_steps)

In [9]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [10]:
# Tạo DataLoader cho tập huấn luyện và kiểm tra
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm  # For progress tracking

# Define the SimpleCNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self._flattened_size = self._get_flattened_size()
        self.fc1 = nn.Linear(self._flattened_size, 128)
        self.fc2 = nn.Linear(128, 2)  # Two classes: Cat and Dog

    def _get_flattened_size(self):
        x = torch.zeros(1, 3, 370, 370)  # Dummy input to calculate size
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        return x.numel()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self._flattened_size)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the ComplexCNN model
class ComplexCNN(nn.Module):
    def __init__(self):
        super(ComplexCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self._flattened_size = self._get_flattened_size()
        self.fc1 = nn.Linear(self._flattened_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 2)

    def _get_flattened_size(self):
        x = torch.zeros(1, 3, 370, 370)  # Dummy input to calculate size
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        return x.numel()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, self._flattened_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Function to train the model
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

# Function to evaluate the model
def evaluate(model, test_loader, criterion, device):
    model.eval()
    test_loss = 0.0
    correct = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    accuracy = correct / len(test_loader.dataset)
    return test_loss / len(test_loader), accuracy

# Set up training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ComplexCNN().to(device)  # Switch to ComplexCNN() if you want to train the complex model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

# Train and evaluate the model
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")





Epoch 1/10


Evaluating: 100%|██████████| 157/157 [00:47<00:00,  3.32it/s]


Train Loss: 0.6718, Test Loss: 0.6195, Test Accuracy: 0.6616
Epoch 2/10


Evaluating: 100%|██████████| 157/157 [00:46<00:00,  3.40it/s]


Train Loss: 0.5602, Test Loss: 0.5652, Test Accuracy: 0.7214
Epoch 3/10


...

Train Loss: 0.0201, Test Loss: 2.1381, Test Accuracy: 0.7040
Epoch 9/10


Evaluating: 100%|██████████| 157/157 [00:45<00:00,  3.48it/s]


Train Loss: 0.0175, Test Loss: 2.2480, Test Accuracy: 0.7194
Epoch 10/10


Evaluating: 100%|██████████| 157/157 [00:45<00:00,  3.48it/s]

Train Loss: 0.0097, Test Loss: 2.2595, Test Accuracy: 0.7150


In [12]:
from torchvision import models

# Define a function to load and fine-tune a pretrained model
def get_pretrained_model():
    # Load ResNet18 pre-trained on ImageNet
    model = models.resnet18(pretrained=True)

    # Freeze all the layers except the final fully connected layer
    for param in model.parameters():
        param.requires_grad = False

    # Modify the final fully connected layer to output 2 classes (Cat and Dog)
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, 2)  # Output layer for 2 classes

    return model

# Load the pretrained model
model = get_pretrained_model().to(device)

# Use CrossEntropyLoss and Adam optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Only optimize the final layer

# Train and evaluate the model again with the pretrained model
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    train_loss = train(model, train_loader, criterion, optimizer, device)
    test_loss, test_accuracy = evaluate(model, test_loader, criterion, device)
    print(f"Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Save the fine-tuned model
torch.save(model.state_dict(), 'pretrained_cnn_model.pth')
print("Pretrained model saved to pretrained_cnn_model.pth")

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 81.8MB/s]


Epoch 1/10


Evaluating: 100%|██████████| 157/157 [00:49<00:00,  3.20it/s]


Train Loss: 0.1515, Test Loss: 0.0763, Test Accuracy: 0.9730
Epoch 2/10


Evaluating: 100%|██████████| 157/157 [00:48<00:00,  3.21it/s]


...

Train Loss: 0.0727, Test Loss: 0.0527, Test Accuracy: 0.9794
Epoch 9/10


Evaluating: 100%|██████████| 157/157 [00:48<00:00,  3.22it/s]


Train Loss: 0.0670, Test Loss: 0.0595, Test Accuracy: 0.9764
Epoch 10/10


Evaluating: 100%|██████████| 157/157 [00:49<00:00,  3.18it/s]

Train Loss: 0.0695, Test Loss: 0.0522, Test Accuracy: 0.9800
Pretrained model saved to pretrained_cnn_model.pth
